In [2]:
import joblib
import numpy as np
import pandas as pd

In [3]:
# data = pd.read_csv('data/final.csv')#合并日期
# data = data.drop(['Latitute','Longtitute'],axis=1)
# data[-3:].to_csv('data/show_data.csv',index=False)

# 导入模型

In [4]:
model_name = './models/multi_xgb.model'
model = joblib.load(model_name)
print(model)

MultiOutputRegressor(estimator=XGBRegressor(base_score=None, booster=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None,
                                            enable_categorical=False,
                                            gamma=None, gpu_id=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None,
                                            max_delta_step=None, max_depth=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=Non...
                                            param_grid={'colsample_bytree': 0.5,
                                                        'gamma': 0

# 处理数据

In [5]:
data = pd.read_csv('data/show_data.csv',parse_dates=[['Crash Time','DATE']])#合并日期
data = data.drop(['STATION','NAME','ID','Vehicle Action Prior to Crash',
                    'Vehicle Travel Directions','X Cooordinate','Y Cooordinate'],axis=1)

In [6]:
for df in [data]:
    df['year'] = df['Crash Time_DATE'].dt.year
    df['month'] = df['Crash Time_DATE'].dt.month
    df['day'] = df['Crash Time_DATE'].dt.day
    df['hour'] = df['Crash Time_DATE'].dt.hour

    df['hour_category'] = np.nan
    df['season'] = np.nan
    df['week'] = np.nan
    
    df.loc[(df.hour <= 9)&(df.hour >= 7), 'hour_category'] = "rush"
    df.loc[(df.hour <= 19)&(df.hour >= 16), 'hour_category'] = "rush"
    df.loc[(df.hour < 16)&(df.hour > 9), 'hour_category'] = "day"
    df.loc[(df.hour < 7)|(df.hour > 19), 'hour_category'] = "night"
    
    df.loc[(df.month <= 3)&(df.month >= 1), 'season'] = 1
    df.loc[(df.month <= 6)&(df.month >= 4), 'season'] = 2
    df.loc[(df.month <=9)&(df.month >= 7), 'season'] = 3
    df.loc[(df.month <=12)&(df.month >=10), 'season'] = 4
    
    df.loc[(df.day <= 7)&(df.day >= 1), 'week'] = 1
    df.loc[(df.day <= 14)&(df.day >= 8), 'week'] = 2
    df.loc[(df.day <=21)&(df.day >= 15), 'week'] = 3
    df.loc[(df.day >=22), 'week'] = 4

In [7]:
Severity = pd.get_dummies(data['Crash Severity'], prefix='Severity', prefix_sep='_')

Injury_Reported = pd.get_dummies(data['Maximum Injury Severity Reported'], prefix='Injury_Reported', prefix_sep='_')

Manner = pd.get_dummies(data['Manner of Collision'], prefix='Manner', prefix_sep='_')

Surface_Condition = pd.get_dummies(data['Road Surface Condition'], prefix='Condition', prefix_sep='_')

Ambient_Light = pd.get_dummies(data['Ambient Light'], prefix='Light', prefix_sep='_')

Weather_Condition = pd.get_dummies(data['Weather Condition'], prefix='Weather', prefix_sep='_')

Ice = pd.get_dummies(data['Ice'], prefix='Ice', prefix_sep='_')

Hour = pd.get_dummies(data['hour_category'], prefix='Hour', prefix_sep='_')

Season = pd.get_dummies(data['season'], prefix='season', prefix_sep='_')

Week = pd.get_dummies(data['week'], prefix='week', prefix_sep='_')


In [8]:
Data_Master = pd.concat([data,
                          Surface_Condition,
                          Ambient_Light,
                          Weather_Condition,
                          Hour,
                          Season,
                          Week
                         ], axis=1)

In [9]:
data = Data_Master.drop(['Crash Severity','Maximum Injury Severity Reported',
                                  'Manner of Collision','Road Surface Condition',
                                  'Ambient Light','Weather Condition','Ice','hour_category','Crash Time_DATE',
                                  'Number of Vehicles','Number of Vehicles','Total Fatal Injuries',
                                  'Total Nonfatal Injuries' ], axis = 1)

In [10]:
file_obj = open("data/attributes.txt")
all_lines = file_obj.readlines()
columns = []
for line in all_lines:
    columns.append(line.strip())
columns.remove('Longtitute')
columns.remove('Latitute')

In [11]:
for column in columns:
    if column not in data.columns:
        data[column] = 0

In [12]:
results = model.predict(data)

In [13]:
results

array([[ 41.941227, -71.27854 ],
       [ 41.92765 , -71.29881 ],
       [ 41.937443, -71.27945 ],
       [ 41.934162, -71.32741 ]], dtype=float32)

In [14]:
# data_item = {}
# Crash_Time_DATE= input("input: Crash Time_DATE: ")
# Crash_Severity = input("input: Crash_Severity: ")
# Maximum_Injury_Severity_Reported = input("input: Maximum Injury Severity Reported: ")
# Number_of_Vehicles = input("input: Number_of_Vehicles: ")
# Total_Fatal_Injuries = input("input: Total_Fatal_Injuries: ")
# Total_Nonfatal_Injuries = input("input: Total_Nonfatal_Injuries: ")
# Manner_of_Collision = input("input: Manner_of_Collision: ")
# Road_Surface_Condition = input("input: Road_Surface_Condition: ")
# Ambient_Light = input("input: Ambient_Light: ")
# Weather_Condition = input("input: Weather_Condition: ")
# PRCP = input("input: PRCP: ")
# SNOW = input("input: SNOW: ")
# TMAX = input("input: TMAX: ")
# TMIN = input("input: TMIN: ")
# Ice = input("input: Ice: ")

In [27]:
Location=pd.DataFrame(results)

In [28]:
Location.columns =['longtitute', 'latitute']

In [29]:
from folium import plugins
import numpy as np 
import pandas as pd 
import folium
import webbrowser
from folium.plugins import HeatMap

In [30]:
import statistics
meanLong = statistics.mean(Location['longtitute'])
meanLat = statistics.mean(Location['latitute'])

In [31]:
m = folium.Map(location=[meanLat, meanLong], zoom_start = 14.5)

In [32]:
LocationM = Location.values

In [35]:
m.add_child(plugins.HeatMap(LocationM, radius=13))